In [1]:
import matplotlib.pyplot as plt
import numpy as np

ref_res={
"D":3.90,
"E":4.36,
"C":8.55,
"EE":4,
"DD":3,
"CC":8}

R  = 8.314
T  = 298.15
B  = 1/(R*T)
ph = np.arange(-14,14,0.1)


def coupled_ddG(ddG0, ddG1, ddG2, res, ph=np.arange(-14,14,0.1),a=1.0):  
    dG1 = deltaG(ddG1, res, T, ph, a)
    dG2 = deltaG(ddG2, res, T, ph, a)    
    
    return (ddG0 +\
            (1/B)*np.log(1+np.exp(-B*dG1)) -\
            (1/B)*np.log(1+np.exp(-B*dG2)))

def deltaG(ddG, res, T=T, ph=np.arange(-14,14,0.1),a=1.0):
    return ddG + a*(R*T*np.log(10)*(ref_res[res]-ph))


# for finding where dG_protein = 0
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx


In [2]:
#  real example D21-D40
#
#         (F11)    6.57   (F01)
#      D21H + D40H  -->  D21- + D40H
#
#  -0.18  |                  | 2.04
#         V                  V
#                  9.42    
#     D21H + D40-  -->  D21- + D40-
#        (F10)             (F00)

# D21-D83
ddG    = [7.98,-15.58,-14.33,9.42]
ddGerr = [0.80,0.97,0.46,0.37]
res1 = "D"
res2 = "D"

# D21-D19
ddG    = [-1.66,-3.17,7.11,9.42]
ddGerr = [0.63,0.31,0.21,0.37]
res1 = "D"
res2 = "D"

# C106-E18 (monomeric)
ddG = [-11.21,15.37,35.39,8.54]
ddGerr = [0.53,0.69,2.34,0.54]
res1 = "C"
res2 = "E"

# C106-E18 (dimer)
ddG = [1.49,31.93,49.13,20.68]
ddGerr = [0.22,0.49,1.61,0.36]
res1 = "C"
res2 = "E"

# D21-D40
ddG    = [6.57,-0.18,2.04,9.42]
ddGerr = [0.55,0.26,0.14,0.37]
res1 = "D"
res2 = "D"

In [3]:
# resample with errors
ddGresample = np.random.normal(np.array(ddG)*1000, np.array(ddGerr)*1000, (100,len(ddG)))

pka1 = []
pka2 = []

for d in ddGresample:
    # compute apparent ddG(pH) due to forward and backwards
    # add peptide deprot to get dG_protein
    # (i.e., ddG = dGprotein - dG_peptide -> ddG + dG_peptide = dG_protein)
    dG1_protein = coupled_ddG(d[0], d[1], d[2],res2,ph, 1.0)  + R*T*np.log(10)*(ref_res[res1]-ph)  
    dG2_protein = coupled_ddG(d[1], d[0], d[3],res1,ph, 1.0)  + R*T*np.log(10)*(ref_res[res2]-ph)
    dG3_protein = coupled_ddG(d[2],-d[0],-d[3],res1,ph,a=-1.0) + R*T*np.log(10)*(ref_res[res2]-ph)  
    dG4_protein = coupled_ddG(d[3],-d[1],-d[2],res2,ph,a=-1.0) + R*T*np.log(10)*(ref_res[res1]-ph)

    # find when dG_protein = 0
    idx1 = find_nearest(dG1_protein,0)
    idx2 = find_nearest(dG2_protein,0)
    idx3 = find_nearest(dG3_protein,0)
    idx4 = find_nearest(dG4_protein,0)
    
    pka1.append(ph[idx1])
    pka2.append(ph[idx2])

print("pka1: %.2f pka2: %.2f" % (np.mean(pka1),np.mean(pka2)))


pka1: 5.43 pka2: 3.88
